CSE 572 Final Project Stock Prediction Using Technical Indicators and Sentiment Analysis: A Binary Classifier
Approach through Data Mining

Dowload the kaggle and Yfinance dataset.

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! kaggle datasets download miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests
! unzip massive-stock-news-analysis-db-for-nlpbacktests

massive-stock-news-analysis-db-for-nlpbacktests.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  massive-stock-news-analysis-db-for-nlpbacktests.zip
replace analyst_ratings_processed.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Display kaggle dataset 

In [ ]:
import pandas as pd
data = pd.read_csv("analyst_ratings_processed.csv");
data.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


Display Y finance Data. Added column to showcase if the stock rose or dipped on a particular day.

In [ ]:
import yfinance as yf
yfin_data1 = yf.download("A", '2011-03-02','2011-03-12')
# Create a new column called 'Price Change' that indicates whether the stock price went up or down by the end of the day
yfin_data1['Price Change'] = ['Up' if yfin_data1['Close'][i] > yfin_data1['Open'][i] else 'Down' for i in range(len(yfin_data1))]
yfin_data1

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Price Change
Date,,,,,,,
2011-03-02,29.020029,29.735336,28.869814,29.656652,26.847456,4721465,Up
2011-03-03,30.128756,30.858370,30.128756,30.629471,27.728123,6092484,Up
2011-03-04,31.008583,33.483547,31.008583,33.440628,30.273010,18664139,Up
2011-03-07,33.090130,33.476395,32.460659,32.811157,29.703154,10355405,Down
2011-03-08,32.911301,33.054363,32.675251,32.882690,29.767908,5445350,Down
2011-03-09,32.653790,32.939915,31.902718,32.939915,29.819708,4869653,Up
2011-03-10,32.446350,32.696709,31.988556,32.103004,29.062078,6788269,Down
2011-03-11,32.238914,32.546494,31.402002,32.432045,29.359955,6208798,Up


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
import nltk
nltk.download('punkt')


def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize text
    tokens = nltk.word_tokenize(text.lower())
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stem words
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

data['text_tokens'] = data['title'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data.head()

,Unnamed: 0,title,date,stock,text_tokens
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,"[stock, hit, 52week, high, friday]"
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,"[stock, hit, 52week, high, wednesday]"
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,"[71, biggest, mover, friday]"
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,"[46, stock, move, friday, midday, session]"
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,"[b, secur, maintain, neutral, agil, technolog,..."


In [ ]:
import gensim.downloader as api
import numpy as np

# Load pre-trained Word2Vec model
w2v_model = api.load('word2vec-google-news-300')

# Convert each word into a Word2Vec embedding

data['text_vectors'] = data['text_tokens'].apply(lambda tokens: np.mean([w2v_model[w] for w in tokens if w in w2v_model.key_to_index], axis=0))

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
data.head()

,Unnamed: 0,title,date,stock,text_tokens,text_vectors
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,"[stock, hit, 52week, high, friday]","[0.005493164, -0.032699585, -0.020721436, 0.12..."
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,"[stock, hit, 52week, high, wednesday]","[0.0785408, -0.03492737, 0.016296387, 0.130126..."
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,"[71, biggest, mover, friday]","[-0.021525064, -0.06656901, 0.010538737, 0.026..."
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,"[46, stock, move, friday, midday, session]","[-0.06889649, 0.006616211, -0.084545895, 0.083..."
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,"[b, secur, maintain, neutral, agil, technolog,...","[-0.014009476, 0.064689636, -0.010437012, 0.13..."
